In [1]:
!pip install upgrade
!pip install pandas-datareader

ERROR: Could not find a version that satisfies the requirement upgrade
ERROR: No matching distribution found for upgrade
  Using cached pandas_datareader-0.9.0-py3-none-any.whl (107 kB)


In [2]:
import MetaTrader5 as mt5
from datetime import datetime
import time
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# estabelecemos a conexão ao MetaTrader 5
# conecte-se ao MetaTrader 5
if not mt5.initialize():
    print("initialize() Falha ao Iniciar seu metra Trade 5")
    mt5.shutdown()
else:
    print('ok')

ok


In [5]:
### Obtendo valores de ativos
#tenho que fazer uma estrutura de repecticao  para ficar gemrado os pontos de entrar e saida
def get_ohlc(ativo, timeframe, n=5):
    ativo = mt5.copy_rates_from_pos(ativo,timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo['Pivot'] = (ativo['high'] + ativo['low'] + ativo['close'])/3
    ativo['R1'] = 2*ativo['Pivot'] - ativo['low']
    ativo['S1'] = 2*ativo['Pivot'] - ativo['high']
    ativo['R2'] = ativo['Pivot'] + (ativo['high'] - ativo['low'])
    ativo['S2'] = ativo['Pivot'] - (ativo['high'] - ativo['low'])
    ativo['R3'] = ativo['Pivot'] + 2*(ativo['high'] - ativo['low'])
    ativo['S3'] = ativo['Pivot'] - 2*(ativo['high'] - ativo['low'])

    
    ativo.set_index('time', inplace = True)
    return ativo

In [10]:
symbol = 'ITUB4'
ativo = (get_ohlc(symbol, mt5.TIMEFRAME_M5))

In [11]:
ativo

,open,high,low,close,tick_volume,spread,real_volume,Pivot,R1,S1,R2,S2,R3,S3
time,,,,,,,,,,,,,,
2021-06-02 16:20:00,31.68,31.70,31.64,31.67,754,1,503400,31.670000,31.700000,31.640000,31.730000,31.610000,31.790000,31.550000
2021-06-02 16:25:00,31.67,31.71,31.67,31.69,661,1,276500,31.690000,31.710000,31.670000,31.730000,31.650000,31.770000,31.610000
2021-06-02 16:30:00,31.69,31.72,31.59,31.60,829,1,742500,31.636667,31.683333,31.553333,31.766667,31.506667,31.896667,31.376667
2021-06-02 16:35:00,31.59,31.65,31.57,31.63,904,1,648600,31.616667,31.663333,31.583333,31.696667,31.536667,31.776667,31.456667
2021-06-02 16:40:00,31.63,31.65,31.61,31.63,225,1,105600,31.630000,31.650000,31.610000,31.670000,31.590000,31.710000,31.550000


In [16]:
#testando se o ativo é valido 
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "Não encontrato")
    mt5.shutdown()
    quit()
else:
    print('ok', symbol)

ok ITUB4


In [19]:
#adicionado symbol se nao existir
if not symbol_info.visible:
    print('Symbol não visivel, tentando adicionar')
    if not mt5.symbol_select(symbol,True):
        print('symbol_select({{}})failed, exit', symbol)
        mt5.shutdown()
        quit()
else:
    print('ok!')

ok!
